In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

### Reading the Dataset

In [3]:
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("./datasets/uci-diabetes.csv"))

In [4]:
#df.toPandas().head()

In [5]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

### Selecting Features and Casting it to required Dtype

In [6]:
columns  = df.columns

dataset = df.select(col(columns[1]).cast('float'),
                    col(columns[2]).cast('float'),
                    col(columns[3]).cast('string'),
                    col(columns[4]).cast('float'),
                   )

### Feature Transformation

In [7]:
indexer = StringIndexer(inputCol="code", outputCol="codeIndex")
encoder = OneHotEncoder(inputCol="codeIndex", outputCol="codeVec")
vectorAssembler = VectorAssembler(inputCols=['value','codeVec'],
                                  outputCol="features")
#normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
scaler = StandardScaler(inputCol="features", outputCol="features_norm")
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, scaler])
final_df = pipeline.fit(dataset).transform(dataset)

### Splitting the Dataset into training data and test data

In [8]:
(training_data, test_data) = final_df.randomSplit([0.8,0.2],seed=1234)

### Building Model

In [9]:
svc = LinearSVC()
param_grid = (ParamGridBuilder()
              .baseOn({svc.labelCol : 'Class'})
              .baseOn([svc.predictionCol, 'pred_svc'])
              .baseOn([svc.featuresCol, 'features_norm'])
              .addGrid(svc.regParam, [0.01,0.1,0,1,10,20,50,100]).build())

evaluator = BinaryClassificationEvaluator(labelCol= 'Class',rawPredictionCol= 'pred_svc')

cv = CrossValidator(estimator=svc, estimatorParamMaps=param_grid, evaluator=evaluator,seed=0,parallelism=2)

### Training the Model

In [10]:
cvModel = cv.fit(training_data)

In [12]:
cvModel.avgMetrics

[0.9994116112970499,
 0.9994116112970499,
 0.9997926006309734,
 0.9995982640326921,
 0.5,
 0.5,
 0.5,
 0.5]

### Evaluating Trained Model Using Different Evaluation Metrices

In [11]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: 'areaUnderPR'})

0.9997693910756901

In [14]:
evaluator.evaluate(cvModel.transform(test_data), {evaluator.metricName: 'areaUnderROC'})

0.9995110821382007

### Best Params

In [15]:
cvModel.bestModel.getRegParam()

0.0

### Saving the Model

In [16]:
path= r'./SavedModel/svc_model'
cvModel.save(path)

### Loading Trained Model

In [17]:
#svc_model= CrossValidatorModel.read().load(path)

### Result Summary

In [1]:
import pandas as pd

summary_= {
    'areaUnderPR' : 0.9997693910756901,
    'areaUnderROC' : 0.9995110821382007
}
summary= pd.Series(summary_)
summary

areaUnderPR     0.999769
areaUnderROC    0.999511
dtype: float64